### **NER**

### spaCy vs Natasha

Загрузка всего необходимого

In [ ]:
!pip install natasha
!pip install datasets
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=3e399f5a94b46ea8ff670f6456ef179a57e25a1ef649b63e676205a5393461b2
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=466c44fa66261ddae0e394bbb7f3ab53c69cbe54db26f41b35ec24b23de894cc
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built docopt intervaltree
     ━━━━━━━━━━━━━━━━━━━━

Загрузим необходимый датасет

In [ ]:
from datasets import load_dataset
dataset = load_dataset("RCC-MSU/collection3")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9301 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2153 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1922 [00:00<?, ? examples/s]

### Напишем функции для обработки датасета:

In [ ]:
from natasha import (Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger,
                     NewsSyntaxParser, NewsNERTagger,NamesExtractor,
                     PER, Doc, ORG, LOC)
import torch
import time
import spacy


# NATASHA
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

# SPACY
# Загрузим предварительно обученную модель для русского языка
nlp = spacy.load("ru_core_news_sm")

names = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]


def natasha_model(test_text):
  formatted_text = ' '.join(test_text)
  doc = Doc(formatted_text)
  doc.segment(segmenter)
  doc.tag_ner(ner_tagger)
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
  doc.parse_syntax(syntax_parser)
  for span in doc.spans:
    span.normalize(morph_vocab)
  doc_spans = doc.spans
  result_list = [[span.type, span.text.replace(' ', '')] for span in doc_spans]
  return result_list


def spacy_model(test_text):
  formatted_text = ' '.join(test_text)
  # Обработаем текст с использованием spaCy
  doc = nlp(formatted_text)
  # Извлечем именованные сущности
  named_entities = [[ent.label_, ent.text] for ent in doc.ents]
  return named_entities

Протестируем модели и выясним их скорость

In [ ]:
import time
test_text = ['Дополнение', ':', 'Д', '.', 'Медведев', 'присвоил', 'звания',
             'сотрудников', 'полиции', 'и', 'переназначил', '14',
             'руководителей', 'УВД', ',', 'ГУВД', 'и', 'МВД',
             'по', 'субъектам', 'РФ', '.']

start_natasha = time.time()
print(natasha_model(test_text))
finish_natasha = time.time()

start_spacy = time.time()
print(spacy_model(test_text))
finish_spacy = time.time()

time_natasha = finish_natasha - start_natasha
time_spacy = finish_spacy - start_spacy
print(time_natasha, time_spacy)

[['PER', 'Д.Медведев'], ['ORG', 'УВД'], ['ORG', 'ГУВД'], ['ORG', 'МВД'], ['LOC', 'РФ']]
[['PER', 'Медведев'], ['ORG', 'УВД'], ['ORG', 'ГУВД'], ['ORG', 'МВД'], ['LOC', 'РФ']]
0.03781938552856445 0.059273481369018555


Предобработаем теги

In [ ]:
tags_list = dataset["test"]["ner_tags"][0]
print(tags_list)
tokens = dataset["test"]["tokens"][0]
print(tokens)
names = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
tags = [names[tag] for tag in tags_list]
print(tags)

[1, 2, 2, 0, 0, 0, 5, 0, 3]
['А', '.', 'Силуанов', 'назначен', 'управляющим', 'от', 'России', 'в', 'МВФ']
['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'B-LOC', 'O', 'B-ORG']


Возьмём тестовый датасет для дальнейшей работы

In [ ]:
import pandas as pd

dataset_dict = {
    "ner_tags": dataset["test"]["ner_tags"],
    "tokens": dataset["test"]["tokens"]
}

df = pd.DataFrame(dataset_dict)

# Посмотрим на первые несколько строк в DataFrame
print(df.head())

names = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

for index, row in df.iterrows():
    tags = [names[tag] for tag in row["ner_tags"]]

    print(f"Example {index + 1}:")
    print("Tokens:", row["tokens"])
    print("Tags:", tags)
    print("\n")


Выходные данные были обрезаны до нескольких последних строк (5000).
Example 923:
Tokens: ['Как', 'сообщают', 'местные', 'жители', ',', 'боевики', 'террористической', 'сети', 'первыми', 'прибыли', 'на', 'место', 'крушения', 'беспилотника', 'и', 'завладели', 'его', 'обломками', '.']
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


Example 924:
Tokens: ['По', 'предварительной', 'информации', 'йеменской', 'полиции', ',', 'в', 'окрестностях', 'Лодера', 'упал', 'БПЛА', 'типа', 'Predator', ',', 'которые', 'широко', 'используются', 'ЦРУ', 'в', 'борьбе', 'с', 'боевиками', 'на', 'территории', 'Пакистана', 'и', 'Афганистана', '.']
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'O']


Example 925:
Tokens: ['Беспилотник', 'может', 'использоваться', 'для', 'уничтожения', 'наземных', 'целей', 'или', 'же', 'выполнять', 'исключительно', 'р

In [ ]:
names = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]
df["tags"] = df["ner_tags"].apply(lambda tags: [names[tag] for tag in tags])

Очистим теги и оставим только самые необходимые:

*   LOC - location
*   PER - person
*   ORG - organization



In [ ]:
import pandas as pd

names = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

def process_row(row):
    tags_list = row["ner_tags"]
    tokens = row["tokens"]
    tags = [names[tag] for tag in tags_list]
    combined_entities = []
    current_entity = {"entity": None, "words": []}
    for token, tag in zip(tokens, tags):
        if tag.startswith("B-"):
            if current_entity["entity"] is not None:
                combined_entities.append(current_entity)
            current_entity = {"entity": tag[2:], "words": [token]}
        elif tag.startswith("I-"):
            if current_entity["entity"] is not None and tag[2:] == current_entity["entity"]:
                current_entity["words"].append(token)
            else:
                combined_entities.append(current_entity)
                current_entity = {"entity": tag[2:], "words": [token]}
        else:  # tag is "O"
            if current_entity["entity"] is not None:
                combined_entities.append(current_entity)
                current_entity = {"entity": None, "words": [token]}

    # Добавляем последнюю сущность в результат
    if current_entity["entity"] is not None:
        combined_entities.append(current_entity)
    # Формируем строку для нового столбца "clean_tags"
    clean_tags = []
    for entity in combined_entities:
        clean_tags.append([entity['entity'], ''.join(entity['words'])])
    return clean_tags

# Применяем функцию process_row к каждой строке и создаем новый столбец "clean_tags"
df["clean_tags"] = df.apply(process_row, axis=1)

# Выводим DataFrame с новым столбцом
df.head()

,ner_tags,tokens,tags,clean_tags
0,"[1, 2, 2, 0, 0, 0, 5, 0, 3]","[А, ., Силуанов, назначен, управляющим, от, Ро...","[B-PER, I-PER, I-PER, O, O, O, B-LOC, O, B-ORG]","[[PER, А.Силуанов], [LOC, России], [ORG, МВФ]]"
1,"[0, 5, 1, 2, 0, 0, 0, 0, 0, 1, 2, 0, 0, 5, 0, ...","[Президент, России, Дмитрий, Медведев, своим, ...","[O, B-LOC, B-PER, I-PER, O, O, O, O, O, B-PER,...","[[LOC, России], [PER, ДмитрийМедведев], [PER, ..."
2,"[0, 0, 0, 0, 5, 1, 2, 2, 0, 0, 5, 0, 3, 4, 4, ...","["", Назначить, министра, финансов, РФ, А, ., С...","[O, O, O, O, B-LOC, B-PER, I-PER, I-PER, O, O,...","[[LOC, РФ], [PER, А.Силуанова], [LOC, РФ], [OR..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[Указ, вступает, в, силу, со, дня, его, подпис...","[O, O, O, O, O, O, O, O, O]",[]
4,"[0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, ...","[Напомним, ,, 16, декабря, 2011г, ., А, ., Сил...","[O, O, O, O, O, O, B-PER, I-PER, I-PER, O, O, ...","[[PER, А.Силуанов], [PER, АлексеяКудрина], [PE..."


### Датасет подготовлен, теперь посмотрим, на что способны модели

In [ ]:
from tqdm import tqdm

# Создаем пустой столбец "natasha" в DataFrame
df["natasha"] = None

# Используем tqdm для отслеживания прогресса
for i in tqdm(range(len(df))):
    df.at[i, "natasha"] = natasha_model(df.at[i, "tokens"])

# Создаем пустой столбец "spacy" в DataFrame
df["spacy"] = None

# Используем tqdm для отслеживания прогресса
for i in tqdm(range(len(df))):
    df.at[i, "spacy"] = spacy_model(df.at[i, "tokens"])

# Выводим DataFrame с новым столбцом
df


100%|██████████| 1922/1922 [00:38<00:00, 49.68it/s]


,ner_tags,tokens,tags,clean_tags,natasha,spacy
0,"[1, 2, 2, 0, 0, 0, 5, 0, 3]","[А, ., Силуанов, назначен, управляющим, от, Ро...","[B-PER, I-PER, I-PER, O, O, O, B-LOC, O, B-ORG]","[[PER, А.Силуанов], [LOC, России], [ORG, МВФ]]","[[PER, А.Силуанов], [LOC, России], [ORG, МВФ]]","[[PER, Силуанов], [LOC, России], [ORG, МВФ]]"
1,"[0, 5, 1, 2, 0, 0, 0, 0, 0, 1, 2, 0, 0, 5, 0, ...","[Президент, России, Дмитрий, Медведев, своим, ...","[O, B-LOC, B-PER, I-PER, O, O, O, O, O, B-PER,...","[[LOC, России], [PER, ДмитрийМедведев], [PER, ...","[[LOC, России], [PER, ДмитрийМедведев], [PER, ...","[[LOC, России], [PER, Дмитрий Медведев], [PER,..."
2,"[0, 0, 0, 0, 5, 1, 2, 2, 0, 0, 5, 0, 3, 4, 4, ...","["", Назначить, министра, финансов, РФ, А, ., С...","[O, O, O, O, B-LOC, B-PER, I-PER, I-PER, O, O,...","[[LOC, РФ], [PER, А.Силуанова], [LOC, РФ], [OR...","[[LOC, РФ], [PER, А.Силуанова], [LOC, РФ], [OR...","[[LOC, РФ], [PER, Силуанова], [LOC, РФ], [ORG,..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[Указ, вступает, в, силу, со, дня, его, подпис...","[O, O, O, O, O, O, O, O, O]",[],[],[]
4,"[0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, ...","[Напомним, ,, 16, декабря, 2011г, ., А, ., Сил...","[O, O, O, O, O, O, B-PER, I-PER, I-PER, O, O, ...","[[PER, А.Силуанов], [PER, АлексеяКудрина], [PE...","[[PER, А.Силуанов], [PER, АлексеяКудрина], [PE...","[[PER, Силуанов], [PER, Алексея Кудрина], [PER..."
...,...,...,...,...,...,...
1917,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[Победителю, предстоит, отработать, на, посту,...","[O, O, O, O, O, O, O, O, B-PER, O, O, O, O, O,...","[[PER, Килпатрика]]","[[PER, Килпатрика]]","[[PER, Килпатрика]]"
1918,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Уже, в, августе, в, городе, вновь, состоится,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],[],[]
1919,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Как, только, Килпатрик, в, сентябре, 2008, го...","[O, O, B-PER, O, O, O, O, O, O, O, O, O, O, O,...","[[PER, Килпатрик], [PER, Бинг]]","[[PER, Килпатрик], [PER, Бинг]]","[[PER, Килпатрик], [PER, Бинг]]"
1920,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ...","[На, время, до, выборов, градоначальником, авт...","[O, O, O, O, O, O, O, B-PER, O, O, O, B-PER, O...","[[PER, Кокрел], [PER, Килпатрике], [LOC, Детро...","[[PER, Кокрел], [PER, Килпатрике], [LOC, Детро...","[[PER, Кокрел], [PER, Килпатрике], [LOC, Детро..."


Вычислим первоначальную точность (берем целиком столбцы и сравниваем их). Данная точность не отобразит всю картину и не даст объективной оценки.

In [ ]:
def compare_columns_natasha(row):
    return row["clean_tags"] == row["natasha"]
# Применяем функцию и вычисляем процент совпадения
percentage_matching_n = df.apply(compare_columns_natasha, axis=1).mean() * 100

def compare_columns_spacy(row):
    return row["clean_tags"] == row["spacy"]
# Применяем функцию и вычисляем процент совпадения
percentage_matching_s = df.apply(compare_columns_spacy, axis=1).mean() * 100

print(f"Accuracy NATASHA: {percentage_matching_n}%")
print(f"Accuracy SPACY: {percentage_matching_s}%")

Accuracy NATASHA: 86.05619146722164%
Accuracy SPACY: 37.929240374609776%


In [ ]:
import pandas as pd

df['true_PER'] = 0
df['true_LOC'] = 0
df['true_ORG'] = 0

# Проходим по каждой строке DataFrame
for index, row in df.iterrows():
    # Проходим по каждой записи в clean_tags текущей строки
    for tag in row['clean_tags']:
        # Извлекаем тип и значение тега
        tag_type, tag_value = tag

        # Увеличиваем соответствующий счетчик в зависимости от типа тега
        if tag_type == 'PER':
            df.at[index, 'true_PER'] += 1
        elif tag_type == 'LOC':
            df.at[index, 'true_LOC'] += 1
        elif tag_type == 'ORG':
            df.at[index, 'true_ORG'] += 1

# Создаем столбцы с начальными значениями natasha
df['natasha_PER'] = 0
df['natasha_LOC'] = 0
df['natasha_ORG'] = 0

# Проходим по каждой строке DataFrame
for index, row in df.iterrows():
    # Проходим по каждой записи в clean_tags текущей строки
    for tag in row['natasha']:
        # Извлекаем тип и значение тега
        tag_type, tag_value = tag

        # Увеличиваем соответствующий счетчик в зависимости от типа тега
        if tag_type == 'PER':
            df.at[index, 'natasha_PER'] += 1
        elif tag_type == 'LOC':
            df.at[index, 'natasha_LOC'] += 1
        elif tag_type == 'ORG':
            df.at[index, 'natasha_ORG'] += 1

# Создаем столбцы с начальными значениями spacy
df['spacy_PER'] = 0
df['spacy_LOC'] = 0
df['spacy_ORG'] = 0

# Проходим по каждой строке DataFrame
for index, row in df.iterrows():
    # Проходим по каждой записи в clean_tags текущей строки
    for tag in row['spacy']:
        # Извлекаем тип и значение тега
        tag_type, tag_value = tag

        # Увеличиваем соответствующий счетчик в зависимости от типа тега
        if tag_type == 'PER':
            df.at[index, 'spacy_PER'] += 1
        elif tag_type == 'LOC':
            df.at[index, 'spacy_LOC'] += 1
        elif tag_type == 'ORG':
            df.at[index, 'spacy_ORG'] += 1

# Выводим получившийся DataFrame
df.head()


,ner_tags,tokens,tags,clean_tags,natasha,spacy,true_PER,true_LOC,true_ORG,natasha_PER,natasha_LOC,natasha_ORG,spacy_PER,spacy_LOC,spacy_ORG
0,"[1, 2, 2, 0, 0, 0, 5, 0, 3]","[А, ., Силуанов, назначен, управляющим, от, Ро...","[B-PER, I-PER, I-PER, O, O, O, B-LOC, O, B-ORG]","[[PER, А.Силуанов], [LOC, России], [ORG, МВФ]]","[[PER, А.Силуанов], [LOC, России], [ORG, МВФ]]","[[PER, Силуанов], [LOC, России], [ORG, МВФ]]",1,1,1,1,1,1,1,1,1
1,"[0, 5, 1, 2, 0, 0, 0, 0, 0, 1, 2, 0, 0, 5, 0, ...","[Президент, России, Дмитрий, Медведев, своим, ...","[O, B-LOC, B-PER, I-PER, O, O, O, O, O, B-PER,...","[[LOC, России], [PER, ДмитрийМедведев], [PER, ...","[[LOC, России], [PER, ДмитрийМедведев], [PER, ...","[[LOC, России], [PER, Дмитрий Медведев], [PER,...",2,2,2,2,2,2,2,2,2
2,"[0, 0, 0, 0, 5, 1, 2, 2, 0, 0, 5, 0, 3, 4, 4, ...","["", Назначить, министра, финансов, РФ, А, ., С...","[O, O, O, O, B-LOC, B-PER, I-PER, I-PER, O, O,...","[[LOC, РФ], [PER, А.Силуанова], [LOC, РФ], [OR...","[[LOC, РФ], [PER, А.Силуанова], [LOC, РФ], [OR...","[[LOC, РФ], [PER, Силуанова], [LOC, РФ], [ORG,...",1,3,3,1,3,3,1,3,3
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[Указ, вступает, в, силу, со, дня, его, подпис...","[O, O, O, O, O, O, O, O, O]",[],[],[],0,0,0,0,0,0,0,0,0
4,"[0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, ...","[Напомним, ,, 16, декабря, 2011г, ., А, ., Сил...","[O, O, O, O, O, O, B-PER, I-PER, I-PER, O, O, ...","[[PER, А.Силуанов], [PER, АлексеяКудрина], [PE...","[[PER, А.Силуанов], [PER, АлексеяКудрина], [PE...","[[PER, Силуанов], [PER, Алексея Кудрина], [PER...",3,0,4,3,0,4,3,0,4


In [ ]:
# Список типов тегов
tag_types = ['LOC', 'PER', 'ORG']

# Словарь для хранения совпадений по каждому типу тегов
match_counts = {tag: {'natasha': 0, 'spacy': 0} for tag in tag_types}

# Проходим по каждой строке DataFrame
for index, row in df.iterrows():
    for tag in tag_types:
        # Сравниваем значения в каждом столбце для natasha и spacy
        if row[f'true_{tag}'] == row[f'natasha_{tag}']:
            match_counts[tag]['natasha'] += 1
        if row[f'true_{tag}'] == row[f'spacy_{tag}']:
            match_counts[tag]['spacy'] += 1

# Общее количество строк
total_count = len(df)

# Рассчитываем точность для каждого типа тегов
accuracy = {tag: {tool: match_counts[tag][tool] / total_count for tool in ['natasha', 'spacy']} for tag in tag_types}

# Выводим значения точности
for tag in tag_types:
    print(f"Accuracy for natasha_{tag}: {accuracy[tag]['natasha']}")
    print(f"Accuracy for spacy_{tag}: {accuracy[tag]['spacy']}")


Accuracy for natasha_LOC: 0.9708636836628513
Accuracy for spacy_LOC: 0.9651404786680541
Accuracy for natasha_PER: 0.9474505723204995
Accuracy for spacy_PER: 0.9349635796045785
Accuracy for natasha_ORG: 0.9640998959417274
Accuracy for spacy_ORG: 0.95369406867846


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Ваши данные
true_PER = df["true_PER"]
true_LOC = df["true_LOC"]
true_ORG = df["true_ORG"]

natasha_PER = df["natasha_PER"]
natasha_LOC = df["natasha_LOC"]
natasha_ORG = df["natasha_ORG"]

spacy_PER = df["spacy_PER"]
spacy_LOC = df["spacy_LOC"]
spacy_ORG = df["spacy_ORG"]

# Функция для вычисления метрик и вывода результатов
def calculate_metrics(true_labels, predicted_labels):
    f1 = f1_score(true_labels, predicted_labels, average='micro')
    return f1

# Вычисление метрик для natasha
a = calculate_metrics(true_PER, natasha_PER)
b = calculate_metrics(true_LOC, natasha_LOC)
c = calculate_metrics(true_ORG, natasha_ORG)

# Вычисление метрик для spacy
d = calculate_metrics(true_PER, spacy_PER)
e = calculate_metrics(true_LOC, spacy_LOC)
f = calculate_metrics(true_ORG, spacy_ORG)

avg_natasha = (a + b + c) / 3
avg_spacy = (d + e + f) / 3
# Вывод усредненных значений
print(f'Average F1 Score_natasha: {avg_natasha}')
print(f'Average F1 Score_spacy: {avg_spacy}')


Average F1 Score_natasha: 0.9608047173083594
Average F1 Score_spacy: 0.9512660423170308
